In [1]:
!git clone https://github.com/huggingface/transformers.git
%cd transformers
!pip3 install --upgrade .
%cd ../

fatal: destination path 'transformers' already exists and is not an empty directory.
/content/transformers
Processing /content/transformers
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at https://github.com/pypa/pip/issues/7555.
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for transformers: filename=transformers-4.16.0.dev0-py3-none-any.whl size=3379183 sha256=9ccb7a4c0b52622dafa93c98f4f9ddbc6fdd1452f89e2194087de204093e3b45
  Stored in directory: /tmp/pip-ephem-wheel-cache-uxtpaomg/wheels/49/62/f4/6730819eed4e6468662b1519bf3bf46419b2335990c77f8767
Successfully built transformers
  Attempti

In [2]:
!pip install underthesea
!pip3 install vncorenlp

!mkdir -p vncorenlp/models/wordsegmenter
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
!wget https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/wordsegmenter.rdr
!mv VnCoreNLP-1.1.1.jar vncorenlp/ 
!mv vi-vocab vncorenlp/models/wordsegmenter/
!mv wordsegmenter.rdr vncorenlp/models/wordsegmenter/

--2022-01-04 09:45:55--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/VnCoreNLP-1.1.1.jar
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 27412575 (26M) [application/octet-stream]
Saving to: ‘VnCoreNLP-1.1.1.jar’

VnCoreNLP-1.1.1.jar 100%[===================>]  26.14M  92.5MB/s    in 0.3s    

2022-01-04 09:45:55 (92.5 MB/s) - ‘VnCoreNLP-1.1.1.jar’ saved [27412575/27412575]

--2022-01-04 09:45:55--  https://raw.githubusercontent.com/vncorenlp/VnCoreNLP/master/models/wordsegmenter/vi-vocab
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting respo

In [3]:
import numpy as np
import tensorflow as tf
import pandas as pd
import pickle
import os
import re
from underthesea import word_tokenize

In [4]:
from transformers import TFAutoModel, AutoTokenizer
phobert = TFAutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base", use_fast=False)

Some layers from the model checkpoint at vinai/phobert-base were not used when initializing TFRobertaModel: ['lm_head']
- This IS expected if you are initializing TFRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFRobertaModel were initialized from the model checkpoint at vinai/phobert-base.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaModel for predictions without further training.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [5]:
tokenizer.pad_token_id

1

In [6]:
tokenizer.cls_token_id

0

In [7]:
df = pd.read_csv('./drive/MyDrive/modified_rating_list.csv', sep=',', encoding='utf-8')

In [8]:
df.head()

,comment,rating
0,treo đầu dê thịt chó hình_ảnh thùng sữa yomost...,1
1,tuyệt với shipper tuyệt_vời bao shipper tâm ng...,5
2,sữa mẹ uống hộp ói mữa ngày tiêu tức ngực chịu...,2
3,tắc_trách sữa yomost uống bé ỉa_chảy ói mửa lu...,1
4,theo sưa hộp lit ngon hơn cảm_giác hộp sữa băn...,4


In [9]:
from vncorenlp import VnCoreNLP
rdrsegmenter = VnCoreNLP("vncorenlp/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

def remove_punctuation(text):
    text = re.sub(r"[\.,\?]+$-", "", text)
    text = text.replace(",", " ").replace("."," ") \
        .replace(";", " ").replace("“", " ") \
        .replace(":", " ").replace("”", " ") \
        .replace('"', " ").replace("'", " ") \
        .replace("!", " ").replace("?", " ") \
        .replace("-", " ").replace("?", " ")
 
    text = text.strip() 

    return text

def word_segment(text): 
    sentences = rdrsegmenter.tokenize(text) 
    text = ""
    for sentence in sentences:
        text += " " + " ".join(sentence)
    return text.strip()

def filter_spam_word(text, replace_dict): 
    words = text.split()
    result = []
    for word in words:
        if replace_dict.get(word, None):
            word = replace_dict[word]
        result.append(word)
    
    if (" ".join(result).count("<unk>") == len(words)):
        return ""
    return text 

In [10]:
df.dropna(subset=['comment', 'rating'], inplace=True)

In [11]:
X = df['comment'].to_numpy()


In [12]:
y = df['rating'].apply(lambda x: int(x))

In [13]:
import tensorflow as tf
import pickle
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [15]:
extractor = TfidfVectorizer(min_df=5, max_df=0.8, max_features=768)
extractor.fit(X_train)

TfidfVectorizer(max_df=0.8, max_features=768, min_df=5)

In [16]:
X_train =  extractor.transform(X_train)
X_val = extractor.transform(X_val)
X_test = extractor.transform(X_test)

In [17]:
model = LinearSVC(C = 0.05,max_iter=1000)
model.fit(X_train, (y_train-1))

LinearSVC(C=0.05)

In [18]:
acc_train = model.score(X_train, y_train-1)
acc_val = model.score(X_val, y_val-1)
acc_test = model.score(X_test, y_test-1)

print(acc_train)
print(acc_val)
print(acc_test)

0.7040686456832024
0.70139591106152
0.7005002510553624


In [19]:
model = LogisticRegression(C = 1,max_iter=3000, multi_class='multinomial',solver='lbfgs')
model.fit(X_train, (y_train-1))

LogisticRegression(C=1, max_iter=3000, multi_class='multinomial')

In [20]:
acc_train = model.score(X_train, y_train-1)
acc_val = model.score(X_val, y_val-1)
acc_test = model.score(X_test, y_test-1)

print(acc_train)
print(acc_val)
print(acc_test)

0.7096709516946395
0.7054406824970653
0.7056236401167872


In [21]:
# model = RandomForestClassifier()
# model = model.fit(X_train, y_train-1)

In [22]:
model = tf.keras.models.Sequential([
    #tf.keras.layers.Input(),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.0)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.0)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation ='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.0))
])

model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['acc'])

In [ ]:
if not os.path.exists("models"):
    os.makedirs("models")

mcp_save = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/tfidf_nn_classify_model.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)  

model.fit(X_train.toarray(), y_train-1,
          epochs=50,
          batch_size=32,
          validation_data=(X_val.toarray(), y_val-1),
          callbacks=[mcp_save])

Epoch 1/50
10755/10755 [==============================] - 46s 4ms/step - loss: 0.8529 - acc: 0.6969 - val_loss: 0.8088 - val_acc: 0.7073
Epoch 2/50
10755/10755 [==============================] - 44s 4ms/step - loss: 0.8034 - acc: 0.7115 - val_loss: 0.8017 - val_acc: 0.7101
Epoch 3/50
10755/10755 [==============================] - 45s 4ms/step - loss: 0.7917 - acc: 0.7154 - val_loss: 0.7971 - val_acc: 0.7117
Epoch 4/50
10755/10755 [==============================] - 45s 4ms/step - loss: 0.7811 - acc: 0.7186 - val_loss: 0.7940 - val_acc: 0.7129
Epoch 5/50
10755/10755 [==============================] - 45s 4ms/step - loss: 0.7710 - acc: 0.7216 - val_loss: 0.7918 - val_acc: 0.7140
Epoch 6/50
10755/10755 [==============================] - 45s 4ms/step - loss: 0.7602 - acc: 0.7252 - val_loss: 0.7919 - val_acc: 0.7144
Epoch 7/50
10755/10755 [==============================] - 49s 5ms/step - loss: 0.7494 - acc: 0.7288 - val_loss: 0.7949 - val_acc: 0.7130
Epoch 8/50
10755/10755 [=================

In [ ]:
model.load_weights("models/tfidf_nn_classify_model.h5")
model.evaluate(X_train.toarray(), y_train-1)
model.evaluate(X_val.toarray(), y_val-1)
model.evaluate(X_test.toarray(), y_test-1)

In [ ]:
X_tokenized = [tokenizer.encode(sent) for sent in X] 

max_length = 0
for sent in X_tokenized:
    if max_length < len(sent):
        max_length = len(sent)

if max_length > 256: 
    max_length = 256 
truncated_X_tokenized = [(sent[:max_length-1] + [tokenizer.eos_token_id]) if (len(sent) > max_length) else sent for sent in X_tokenized]
padded_X_tokenized = np.array([sent + [tokenizer.pad_token_id]*(max_length-len(sent)) for sent in truncated_X_tokenized]) 
attention_mask = np.where(padded_X_tokenized == tokenizer.pad_token_id , 0 , 1)

print(padded_X_tokenized.shape)
print(attention_mask.shape)

In [ ]:
label = np.array(df['rating'].tolist())

In [ ]:
dataset = tf.data.Dataset.from_tensor_slices((padded_X_tokenized, attention_mask, label)).batch(32) 
print(tf.data.experimental.cardinality(dataset))

In [ ]:

y = []
extracted_features = []
for X_i, attention_map_i, y_i in dataset:
    features = phobert(X_i, attention_map_i)
    final_features = features[0][:,0,:] 
    
    extracted_features.append(final_features.numpy())
    y.append(y_i.numpy())
    
extracted_features = np.concatenate(extracted_features)
y = np.concatenate(y) 
print(extracted_features.shape, y.shape)

In [ ]:

with open('data/bert_extracted_review_data.pkl', 'wb') as outfile:
    pickle.dump((extracted_features, y), outfile, pickle.HIGHEST_PROTOCOL)

In [ ]:
with open("data/bert_extracted_review_data.pkl","rb") as infile:
    X,y = pickle.load(infile)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=1)

In [ ]:
model = LinearSVC(C = 0.005,max_iter=1000)
model.fit(X_train, (y_train-1))

In [ ]:
acc_train = model.score(X_train, y_train-1)
acc_val = model.score(X_val, y_val-1)
acc_test = model.score(X_test, y_test-1)

print(acc_train)
print(acc_val)
print(acc_test)

In [ ]:
model = LinearSVC(C=0.01, max_iter=1000)
model.fit(X_train, np.where((y_train-1)>=4,0,1))


acc_train = model.score(X_train, np.where((y_train-1)>=4,0,1))
acc_val = model.score(X_val, np.where((y_val-1)>=4,0,1))
acc_test = model.score(X_test, np.where((y_test-1)>=4,0,1))

print(acc_train)
print(acc_val)
print(acc_test)

In [ ]:
model = LogisticRegression(C = 0.03,max_iter=3000, multi_class='multinomial',solver='lbfgs')
model.fit(X_train, (y_train-1))

In [ ]:
acc_train = model.score(X_train, y_train-1)
acc_val = model.score(X_val, y_val-1)
acc_test = model.score(X_test, y_test-1)

print(acc_train)
print(acc_val)
print(acc_test)

In [ ]:
model = RandomForestClassifier()
model = model.fit(X_train, y_train-1)

In [ ]:
acc_train = model.score(X_train, y_train-1)
acc_val = model.score(X_val, y_val-1)
acc_test = model.score(X_test, y_test-1)

print(acc_train)
print(acc_val)
print(acc_test)

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(768),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.0)),
    tf.keras.layers.Dense(256, activation='relu', kernel_regularizer=tf.keras.regularizers.L2(0.0)),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(5, activation ='softmax', kernel_regularizer=tf.keras.regularizers.L2(0.0))
])

model.compile(optimizer = tf.keras.optimizers.Adam(1e-4),
              loss=tf.keras.losses.sparse_categorical_crossentropy,
              metrics=['acc'])

In [ ]:
if not os.path.exists("models"):
    os.makedirs("models")

mcp_save = tf.keras.callbacks.ModelCheckpoint(
    filepath="models/bert_nn_classify_model.h5",
    save_weights_only=True,
    monitor='val_acc',
    mode='max',
    save_best_only=True)  

history = model.fit(X_train, y_train-1,
          epochs=50,
          batch_size=32,
          validation_data=(X_val, y_val-1),
          callbacks=[mcp_save])

In [ ]:
model.load_weights("models/bert_nn_classify_model.h5")
model.evaluate(X_train, y_train-1)
model.evaluate(X_val, y_val-1)
model.evaluate(X_test, y_test-1)

In [ ]:
import matplotlib.pyplot as plt
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'r', label='Training accuracy')
plt.plot(epochs, val_acc, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.legend(loc=0)
plt.figure()


plt.show()

In [ ]:
class Metrics():


    @staticmethod
    def computeMSE(y_true, y_pred):
        return np.mean((y_true-y_pred)**2)

    @staticmethod
    def computeMAE(y_true, y_pred):
        return np.mean(np.abs(y_true-y_pred))

    @staticmethod
    def computeSIA(y_true, y_pred, eps=2):
        error = np.abs(y_true - y_pred)
        res = np.mean(((error - eps) < 0 ) & (y_pred >= 0))
        return res

In [ ]:
total_y_pred = tf.argmax(model.predict(X_test), axis=1)
total_y = y_test-1

confusion_matrix = tf.math.confusion_matrix(total_y, total_y_pred).numpy()



import matplotlib.pyplot as plt

import itertools
label_dict_test = {i+1:i for i in range(5)}



def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1, keepdims = True)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

class_names = label_dict_test.keys()

plt.figure(figsize=(10,10))
plot_confusion_matrix(confusion_matrix, classes=class_names, normalize=True,
                      title= None, cmap= plt.cm.Reds)

plt.show()

In [ ]:
Metrics.computeSIA(total_y, total_y_pred, eps=2)